### How To Use
- 2 main sections.
  - 1: Calculate RMSF / RMSD
  - 2: Create Graphs
- First find data.
  - Make sure to run the functions that create the data required for whatever graphs you want to make
  - Can also save / load data from text files, so only need to run heavy duty analysis once
- Final is graphs
  - Check which data is needed for each graph, and what section it is calculated in

### Setup Libraries

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis.rms import RMSD
import prody as prd # really annoying to install. Have to put directly into library folder.
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
from matplotlib.backends.backend_pdf import PdfPages
from numpy import * 
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


#### Set the run types being observed, # of runs for each run type, and RPI
Terminology:
- Run Type: what the MD simulation is modeling. Ex: wild type protein, res109 mutated to W, ect.
- Runs: which run of a run type, as each run type would be simulated 3+ times to improve data accuracy.

Folder Setup:
- Each run type should be stored in a folder that lines up with the name put in run_types list.
- Then each run_type folder should have another set of folders that hold each different run. Each folder should have the same name that was put into run_count list.

In [ ]:
run_types = ['WT_v11','S440W_v1'] # add the name of each 
run_count = ['Run1','Run2','Run3'] # add the name of each 
frames = ["{:04}".format(i) for i in range(1,401)] # change last number based on number of frames + 1
sim_length = 200 #how long the simulation is in nanoseconds

456_SAH ['SAH.N1_456.ND2', '456.ND2_SAH.O1', '456.ND2_SAH.S1', '456.ND2_SAH.N1', 'SAH.N4_456.ND2', 'SAH.N4_456.OD1', '456.ND2_SAH.N5', 'SAH.O5_456.OD1', '456.ND2_SAH.O5', 'SAH.N1_456.OD1']


## 1) Calculate Mean Square Fluctuation (RMSF) & RMSD
*Finds the RMSF of each alpha carbon (or how much each alpha carbon fluctuated over the course of the simulation)*

*Finds the RMSD for each frame (how much each frame changed over the course of the simulation)*

In [ ]:
def evalDCD(currMutation, currRun, ifRMSF = True, reference = [], focus=[]):
    """Takes in a specific MD sim and returns the location of each alpha carbon at each frame, and the mean square
    fluction (RMSF) of each alpha carbon

    Args:
        currMutation (str): which run type is being looked at (ex: WT_v3)
        currRun (str): which version of that run type (ex: Run 1)

    Returns:
        list of ensembles: list of the location of each alpha carbon for each fram
        list of ints: list of the RMSF of each alpha carbon
    """

    # Read in the pdb / dcd asn en ensemble
    # MAY NEED TO EDIT SO IT POINTS AT CORRECT FILES
    pdb_dir = './' + currMutation + '/' + currRun + '/init_coords/tog_step3_input.pdb' #input PDB of sim w/o water/ions
    structure = prd.parsePDB(pdb_dir)
    
    dcd_dir = './' + currMutation + '/' + currRun + '/init_coords/tog_step5.dcd' #the no water, complete dcd
    ensemble = prd.parseDCD(dcd_dir)    
    
    
    # select what referencec is for superposition
    if(reference == [] or reference == ""):
        ensemble.setAtoms(structure.select("ca"))
    elif(isinstance(reference, str)):
        ensemble.setAtoms(f'resname {reference} and name "C[0-9]*"')
    else:
        selection = "ca resnum"
        for ref in reference:
            selection = selection + " " + f"{ref[0]}to{ref[1]}"
        ensemble.setAtoms(structure.select(selection))
    ensemble.setCoords(structure)
    ensemble.superpose()

    # select what RMSD is done of
    if(focus == [] or focus == ""):
        ensemble.setAtoms(structure.calpha)
    elif(focus == "+"):
        ensemble.setAtoms(structure)
    elif(isinstance(focus, str)):
        if(focus.endswith("+")):
            ensemble.setAtoms(structure.select(f'resname {focus[0:-1]}'))
        else:
            ensemble.setAtoms(structure.select(f'resname {focus} and name "C[0-9]*"'))
    else:
        selection = "ca resnum"
        for ref in focus:
            selection = selection + " " + f"{ref[0]}to{ref[1]}"
        ensemble.setAtoms(structure.select(selection))

    

    rmsf = []
    if(ifRMSF): # if finding RMSF
        rmsf = ensemble.getRMSFs()
    else: # if finding RMSD
        rmsf = ensemble.getRMSDs()
    
    # Superpose the ensemble to the center of frame
    ensemble.iterpose()
    
    return ensemble, rmsf

def average_runs(run_types, run_count, metric_data):
    data = {}
    
    for run_type in run_types:
        
        temp = []
        for run in run_count:
            temp.append(metric_data[run_type][run])

        # Zip the lists together
        zipped_lists = np.swapaxes(temp,0, 1)

        # Calculate the average of each group of elements
        average_list = [sum(items) / len(items) for items in zipped_lists]

        data[run_type] = average_list
    
    return data

#### Finds RMSF

In [ ]:
""" Goes through and stores the RMSF of each alpha carbon and the ensemble of each MD sim
 The allRMSF/Ensemble dictionaries structure --> Dim 1: run type (mutation/WT) Dim 2: which run (number)
 These two dictionaries are then put into the metricdata dictionary """

allRMSFs = {}
allEnsemble = {}

# what protein is superposed to. The template
template = [39,65],[109,156],[175 ,256],[300,336 ],[448,485], [572, 598] # can either be [[res1, res2], [res3, res4]], or empty list (whole protein)

# what section the RMSD is calculated for
selection = "" # can either be [[res1, res2], [res3, res4]], lignad, ligand+ (all atoms), + (entire protein), empty list (whole protein Ca).

for run_type in run_types:
    print(run_type)
 
    if run_type not in allRMSFs:
        allRMSFs[run_type] = {}
 
    if run_type not in allEnsemble:
        allEnsemble[run_type] = {}
    
    for run in run_count:
        print(run)
        ensemble, rmsf = evalDCD(run_type, run, True, template, selection)
        allRMSFs[run_type][run] = rmsf
        allEnsemble[run_type][run] = ensemble

# finds averaged RMSFs
allAveragedRMSFs = average_runs(run_types, run_count, allRMSFs)

#### Finds RMSD

In [ ]:
""" Find the RMSD for the entire protein, a part of the protein or even a ligand!"""

allRMSDs = {}

# what protein is superposed to. The template
template = [[428, 440], [462,492], [506, 517], [573, 598], [604, 623]] # can either be [[res1, res2], [res3, res4]], or empty list (whole protein)

# what section the RMSD is calculated for
selection = "SAH" # can either be [[res1, res2], [res3, res4]], lignad, empty list (whole protein)

for run_type in run_types:
    print(run_type)
 
    if run_type not in allRMSDs:
        allRMSDs[run_type] = {}
    
    for run in run_count:
        print(run)
        ensemble, rmsd = evalDCD(run_type, run, False, template, selection)
        allRMSDs[run_type][run] = rmsd


allAveragedRMSDs = average_runs(run_types, run_count, allRMSDs)
print(allRMSDs)

#### Save and Load (Does both RMSF/RMSD)

In [ ]:
# Saving
currDir = "./data_storage/WT_S440W/rmsd_SAH/" # recommend making a subfolder system. Add in name here

for run_type in run_types:

    temp = allRMSFs[run_type] # CHANGE WHAT DATA IS SO SAVES CORRECT VALUE
    file = currDir + run_type.split("_")[0] + ".txt"
    
    with open(file, "w") as w:
        for run in run_count:
            for ind, temp2 in enumerate(temp[run]):
                if ind != 0:
                    w.write(",")
                w.write(str(temp2))
            w.write("\n")

In [ ]:
# loading
currDir = "./data_storage/WT_G196F/rmsd_sah/" # recommend making a subfolder system. Add in name here

def read(run_types, currDir):
    data = {}
    for run_type in run_types:
        data[run_type] = {}
        file = currDir + run_type.split("_")[0] + ".txt"

        with open(file, "r") as w:
            temp = w.read().split("\n")[0:-1]

            for ind, run in enumerate(run_count):
                temp2 = []
                t2 = temp[ind].split(",")
                for t3 in t2:
                    temp2.append(np.float64(t3))
                
                
                data[run_type][run] = temp2
    return data

# edit if rmsd or rmsf
template = []
selection = []
allRMSDs = read(run_types, currDir)
allAveragedRMSDs = average_runs(run_types, run_count, allRMSDs)

print(allRMSDs)

{'WT_v11': {'Run1': [np.float64(1.079700016419334), np.float64(0.7154684624050804), np.float64(1.7311749270330523), np.float64(1.8357526942177522), np.float64(1.6303580236731745), np.float64(1.785244784316241), np.float64(2.181559901862514), np.float64(1.7261343357354744), np.float64(2.230112904675086), np.float64(1.7998609741832425), np.float64(2.0337951706613993), np.float64(1.8490090846656224), np.float64(2.017462598076771), np.float64(1.914086773828199), np.float64(2.0209215590377654), np.float64(1.8512157903793733), np.float64(2.193053549541873), np.float64(2.0424811013829225), np.float64(1.9801425396085224), np.float64(1.957529757809641), np.float64(2.807309214251821), np.float64(2.1620254606970897), np.float64(2.0995040478230225), np.float64(2.021900628030183), np.float64(2.8754961214180192), np.float64(2.7074324565342045), np.float64(2.707817735311312), np.float64(2.617805226519825), np.float64(2.6150049939752398), np.float64(2.8365454966873624), np.float64(2.5615173788878463),

#### Save Aligned Trajecotry

In [ ]:
for run_type in run_types:
    print(run_type)
     
    for run in run_count:
        print(run)
        prd.writeDCD(f"{run_type}/{run}/init_coords/tog_step3_aligned_SAH_pocket.dcd", allEnsemble[run_type][run])

## 2) Create Plots

In [ ]:
# this function will insert annotations into an residue based graph
def annotation(ax, anno_loc, text_loc=-1.3):

    descSize = 10

    ax.axvspan(1, 35, color='#FFA3A3', alpha=0.3)
    ax.annotate('Ser\nRich',
            xy=(2, -0.1+text_loc),
            horizontalalignment='left', verticalalignment='top',
            fontsize=descSize)

    ax.axvspan(48, 296, color='#7CAED1', alpha=0.3)
    ax.axvspan(296, 338, color='#7CAED1', alpha=0.3)
    ax.annotate('Catalytic\nDomain',xy=(120, -0.1+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=descSize)
    ax.annotate('Hinge\nRegion',xy=(298, -0.1+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=8)

    
    ax.axvspan(338, 381, color='#C770CE', alpha=0.3)
    ax.axvspan(381, 394, color='#C770CE', alpha=0.3)
    ax.axvspan(394, 412, color='#C770CE', alpha=0.3)
    ax.annotate('Linker\nRegion',xy=(357, -0.1+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=descSize)
    ax.annotate('L1',xy=(352, 0.0+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=8)
    ax.annotate('L2',xy=(382, 0.0+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=8)
    ax.annotate('L3',xy=(397, 0.0+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=8)

    ax.axvspan(412, 644, color='#B79900', alpha=0.3)
    ax.annotate('Regulatory\nDomain',xy=(500, -0.1+text_loc),horizontalalignment='left', verticalalignment='top',fontsize=descSize)

    #resss = [94,95,127,129,154,156,157,158,159,174,175,195,197,201,202,203,204,210,213,216,217,226,321] # fad
    resss = [365, 435, 438, 439, 453, 471, 456, 461, 464, 481, 482, 483, 484, 485, 506, 560, 573] # sah

    ress_place = np.full(len(resss), anno_loc)
    ax.scatter(resss, ress_place, color="red", label="SAH Pocket Residues")


# will take in values and create a rolling average from it
def rolling_average(values):
    length = len(values)
    new_values = []
    for ind in range(len(values)):
        min = ind-5
        max = ind+5
        if(min < 0):
            min = 0
        if(max >= length):
            max = length-1
        new_values.append(np.average(values[min:max+1]))

    return new_values


In [ ]:
# this function will insert annotations into an residue based graph
def annotation2(ax, anno_loc, text_loc=-1.3):

    descSize = 10
    #alignedRanges = [39,65],[109,156],[206,256],[300,377],[448,622]
    alignedRanges = [39,65],[109,156],[175 ,256],[300,336 ],[448,485], [572, 598]
    #alignedRanges = [124,155],[175,196],[208,256],[317,321]
    #alignedRanges = [[199,205]]

    for align in alignedRanges:
        ax.axvspan(align[0], align[1], color='#FFA3A3', alpha=0.3)

    ax.scatter([-10], [-10], color="#FFA3A3", label="Low RMSF Regions")
    
    #resss = [94,95,127,129,154,156,157,158,159,174,175,195,197,201,202,203,204,210,213,216,217,226,321]
    resss = [365, 435, 438, 439, 453, 471, 456, 461, 464, 481, 482, 483, 484, 485, 506, 560, 573]
    ress_place = np.full(len(resss), anno_loc)
    ax.scatter(resss, ress_place, color="red", label="SAH Pocket Residues")


#### Plot average RMSF of all residues for all run types
*Requires allAveragedRMSFs, which is calculated in section 2*

In [ ]:
# Colors
colors = ["#D4A017", "#008080", "#708090", "#691751"] # ! IF YOU WANT TO PUT COLORS, PUT HERE! SAME ORDER AS RUN_TYPE!
fig, ax = plt.subplots(figsize=(20, 8))

titleSize = 32
labelSize = 25

# plot setting
ax.set_title('Average RMSF Aligned to Low RMSF Regions',fontsize=titleSize)
ax.set_xlabel('Residue Number', fontsize=labelSize)
ax.set_ylabel('RMSF (Å)', fontsize=labelSize)


# show residue with RMSF greater than 42
res_count = 0
for run_type in allAveragedRMSFs.keys():
    print(f"{run_type} residues with RMSF greater than 7")
    residue_list = [(x, i+1) for i, x in enumerate(allAveragedRMSFs[run_type]) if x > 7]
    print(residue_list,"\n")

    if res_count < len(allAveragedRMSFs[run_type]):
        res_count = len(allAveragedRMSFs[run_type])

# setup x values (residue)
res_numbers = range(1,res_count+1)

# Plot the data with the specified colors
for (series, values), color in zip(allAveragedRMSFs.items(), colors):
    ax.plot(res_numbers, values, label=series.split("_")[0], color=color)



# Adjust x-ticks for 640 residues, spaced by 50
Labels = range(0,res_count+1,25)
ax.set_xticks(Labels[1:])
ax.set_ylim(bottom=0, top=6)
#ax.set_xlim(left=565, right=605)
annotation(ax, 0.05, 0.45)
#plt.axvline(x=436, color='red', linestyle='--', linewidth=1)
#plt.axvline(x=439, color='red', linestyle='--', linewidth=1)
#plt.axvline(x=440, color='red', linestyle='--', linewidth=1)
#plt.axvspan(199, 205, color='red', alpha=0.3)

# Add legend
ax.legend(loc='upper right', shadow=True, ncol=4, fontsize=18)

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.style.context("seaborn-talk")
plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_rmsf.png', format='png', bbox_inches='tight') 


#### Plots Each Runs RMSF Seperately
*Requires allRMSFs from Section 2*

*If you want all RMSFS on one graph, put all to True. If you want to just have one run type, put all to False, then put name of run type into run_type graph*

In [ ]:
ci = 0

# decide what to graph
all = True # if all run types on one graph - true, else false
run_type_graph = "WT_v11" # if just one run type, put the name of it here

if all:
    colors = ["#F7BD2B", "#C78F01", "#5A440B", "#42DBDB", "#019E9E", "#002929"]
    toGraph = allRMSFs # graph everything
else:
    colors = ["#F7BD2B", "#C78F01", "#5A440B"] 
    #colors = ["#42DBDB", "#019E9E", "#002929"] 
    toGraph = allRMSFs[run_type_graph] # graph just WT_v11

fig, ax = plt.subplots(figsize=(20, 8)) # move in to make seperate graphs, out for same

titleSize = 32 
labelSize = 25
legendSize = 18

# plot setting
if all:
    ax.set_title(f'RMSF of Each Run',fontsize=titleSize)
else:
    ax.set_title(f'Average RMSF of {run_type_graph.split("_")[0]}',fontsize=titleSize)

ax.set_xlabel('Residue Number', fontsize=labelSize)
ax.set_ylabel('RMSF (Å)', fontsize=labelSize)

# Iterature through each run of each run type and display RMSF
for (run_name, run_type) in toGraph.items():

    # Plot the data with the specified colors
    if(isinstance(run_type, dict)):
        for (series, values) in run_type.items():
            res_numbers = range(1,len(values)+1)
            ax.plot(res_numbers, values, label=f"{run_name.split("_")[0]}_{series[-1]}", color=colors[ci])
            ci = ci + 1
    else:
        res_numbers = range(1,len(run_type)+1)
        ax.plot(res_numbers, run_type, label=f"{run_name}", color=colors[ci])
        ci = ci + 1


# Add legend
annotation2(ax, 0.05, 0.55)
ax.legend(loc='upper right', shadow=True, ncol=4, fontsize=18)

# Adjust x-ticks for 640 residues, spaced by 50
Labels = range(0,len(values)+1,1)
ax.set_xticks(Labels[1:])
ax.set_ylim(bottom=-0, top=7)
ax.set_xlim(left=194, right=210)
#plt.axvspan(199, 205, color='red', alpha=0.3)

#plt.axvline(x=436, color='red', linestyle='--', linewidth=1)
#plt.axvline(x=439, color='red', linestyle='--', linewidth=1)
#plt.axvline(x=440, color='red', linestyle='--', linewidth=1)
plt.style.context("seaborn-talk")

if all:
    plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_all_runs_rmsf.png', format='png', bbox_inches='tight') # move in to make seperate graphs, out for same 
else:
    plt.savefig(f'Plots/{run_type_graph.split("_")[0]}_all_runs_rmsf.png', format='png', bbox_inches='tight') # move in to make seperate graphs, out for same 


#### Plots deltaRMSF (type - WT)

In [ ]:
# Colors
colors = ["#D4A017", "#008080", "#708090", "#691751"] # ! IF YOU WANT TO PUT COLORS, PUT HERE! SAME ORDER AS RUN_TYPE!
fig, ax = plt.subplots(figsize=(20, 8))

titleSize = 32 
labelSize = 25
legendSize = 18

# plot setting
ax.set_title('Average ΔRMSF (S440W - WT)', fontsize=titleSize)
ax.set_xlabel('Residue Number', fontsize=labelSize)
ax.set_ylabel('RMSF (Å)', fontsize=labelSize)


# show residue with RMSF greater than 42
res_count = 0
for run_type in allAveragedRMSFs.keys():
    print(f"{run_type} residues with RMSF greater than 7")
    residue_list = [(x, i+1) for i, x in enumerate(allAveragedRMSFs[run_type]) if x > 7]
    print(residue_list,"\n")

    if res_count < len(allAveragedRMSFs[run_type]):
        res_count = len(allAveragedRMSFs[run_type])

# setup x values (residue)
res_numbers = range(1,res_count+1)

# find delta
baseKey = "WT_v11"
allDeltaRMSFs = {}
for run_type in allAveragedRMSFs.keys():
    allDeltaRMSFs[run_type] = []
    for resInd in range(len(allAveragedRMSFs[run_type])):
        allDeltaRMSFs[run_type].append(allAveragedRMSFs[run_type][resInd] - allAveragedRMSFs[baseKey][resInd])


# Plot the data with the specified colors
for (series, values), color in zip(allDeltaRMSFs.items(), colors):
    ax.plot(res_numbers, values, label=series, color=color)

# Add legend
annotation(ax, -0.98, -0.7)
ax.legend(loc='upper right', shadow=True, ncol=4, fontsize=legendSize)

# Adjust x-ticks for 640 residues, spaced by 50
Labels = range(0,res_count+1,25)
ax.set_xticks(Labels[1:])
ax.set_ylim(bottom=-1, top=2)
#ax.set_xlim(left=330, right=370)
#plt.axvline(x=436, color='red', linestyle='--', linewidth=1)
#plt.axvline(x=439, color='red', linestyle='--', linewidth=1)
#plt.axvline(x=440, color='red', linestyle='--', linewidth=1)
#plt.axvspan(199, 205, color='red', alpha=0.3)
plt.style.context("seaborn-talk")

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_delta_rmsf.png', format='png', bbox_inches='tight') 


#### Plots Each Runs RMSD Seperately
*Requires allRMSDs from Section 2*

*If you want all RMSDS on one graph, put all to True. If you want to just have one run type, put all to False, then put name of run type into run_type graph*

In [ ]:
# Colors
# decide what to graph
ci = 0
all = True # if all run types on one graph - true, else false
run_type_graph = "" # if just one run type, put the name of it here
fig, ax = plt.subplots(figsize=(20, 8))
averaged = 10

titleSize = 32 
labelSize = 25
legendSize = 18

if all:
    colors = ["#F7BD2B", "#C78F01", "#5A440B", "#42DBDB", "#019E9E", "#002929"]
    toGraph = allRMSDs # graph everything

else:
    colors = ["#D4A017", "#008080", "#708090", "#691751"] 
    toGraph = allRMSDs[run_type_graph] # graph just WT_v11fig, ax = plt.subplots(figsize=(20, 8))

# plot setting
#ax.rcParams.update({'font.size': 30})
if selection == []:
    ax.set_title('Rolling RMSD Aligned to FAD Pocket', fontsize=titleSize)
if isinstance(selection, str):
    ax.set_title(f'Rolling RMSD for {selection} Aligned to Pocket', fontsize=titleSize)
elif len(selection) >= 2:
    ax.set_title(f'Rolling RMSD for {selection[0]}-{selection[1]} Aligned to FAD Pocket', fontsize=titleSize)
else:
    ax.set_title(f'Rolling SAH RMSD Aligned to SAH Pocket', fontsize=titleSize)

ax.set_xlabel('Time (ns)', fontsize=labelSize)
ax.set_ylabel('RMSD (Å)', fontsize=labelSize)


# Plot the data with the specified colors
timesteps = np.linspace(0, sim_length, len(list(list(allRMSDs.values())[0].values())[0])+1)[1:]
for (run_name, run_type) in toGraph.items():

    # Plot the data with the specified colors
    if(isinstance(run_type, dict)):
        for (series, values) in run_type.items():
            new_values = rolling_average(values)
            ax.plot(timesteps, new_values, label=f"{run_name.split("_")[0]}_{series[-1]}", color=colors[ci])
            ci = ci + 1
    else:
        ax.plot(timesteps, run_type, label=f"{run_name.split("_")[0]}", color=colors[ci])
        ci = ci + 1

# Add legend
ax.legend(loc='upper left', shadow=True, ncol=4, fontsize=legendSize)

ax.set_ylim(bottom=0, top=10)
plt.style.context("seaborn-talk")

# Save the figure as a vectorized PDF or SVG for Adobe Illustrator
plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_all_runs_rmsd.png', format='png', bbox_inches='tight')  # For SVG format

#### Plots RMSD
*Requires allRMSDs which is calculated in section 2*

In [ ]:
# Colors
colors = ["#D4A017", "#008080", "#708090", "#691751"] # ! IF YOU WANT TO PUT COLORS, PUT HERE! SAME ORDER AS RUN_TYPE!
fig, ax = plt.subplots(figsize=(20, 8))

titleSize = 32 
labelSize = 25
legendSize = 18

# plot setting
#ax.rcParams.update({'font.size': 30})
if selection == []:
    ax.set_title('Average SAH RMSD using center structure', fontsize=titleSize)
if isinstance(selection, str):
    ax.set_title(f'Average RMSD for {selection} using center structure', fontsize=titleSize)
elif len(selection) >= 2:
    ax.set_title(f'Average RMSD for {selection[0]}-{selection[1]} using center structure', fontsize=titleSize)
else:
    ax.set_title(f'Average SAH RMSD Aligned to SAH Pocket', fontsize=titleSize)

ax.set_xlabel('Time (ns)', fontsize=labelSize)
ax.set_ylabel('RMSD (Å)', fontsize=labelSize)


# Plot the data with the specified colors
for (series, values), color in zip(allAveragedRMSDs.items(), colors):
    ax.plot(np.linspace(0, sim_length, len(values)+1)[1:], values, label=series.split("_")[0], color=color)

# Add legend
ax.legend(loc='upper right', shadow=True, ncol=4, fontsize=legendSize)
plt.style.context("seaborn-talk")

ax.set_ylim(bottom=0, top=8)
plt.savefig(f'Plots/{run_types[0].split("_")[0]}_{run_types[1].split("_")[0]}_rmsd.png', format='png', bbox_inches='tight')  # For SVG format